## A Test Notebook - Recreate Lesson 1

### Boilerplate Imports

In [12]:
%matplotlib inline

from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

### Bring in the data
path to data sample and main

In [13]:
 path = "data/dogscats/"
#path = "data/dogscats/sample/"

### Utility Functions
utils.py contains useful functions that we use

In [14]:
import utils; reload(utils)
from utils import plots

### Bring in the pretrained model and scripts used to work with it
batch size should be in a power of 2 (2,4,8,16,32,64 etc)

In [15]:
batch_size = 64
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [16]:
vgg = Vgg16()
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size)
vgg.finetune(batches)
vgg.fit(batches, val_batches,nb_epoch=1)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/1
23000/23000 [==============================] - 579s - loss: 0.1280 - acc: 0.9663 - val_loss: 0.0554 - val_acc: 0.9840


In [17]:
imgs,labels = next(batches)
vgg.predict(imgs, True)

(array([1.    , 0.9658, 0.7891, 1.    , 1.    , 1.    , 1.    , 0.9926, 1.    , 1.    , 1.    ,
        1.    , 1.    , 0.7257, 1.    , 1.    , 0.9563, 1.    , 1.    , 1.    , 1.    , 1.    ,
        1.    , 1.    , 1.    , 1.    , 1.    , 0.9999, 0.9999, 1.    , 1.    , 1.    , 1.    ,
        1.    , 1.    , 0.9357, 0.9978, 1.    , 1.    , 1.    , 0.6101, 0.9991, 1.    , 1.    ,
        1.    , 1.    , 1.    , 1.    , 0.9944, 1.    , 1.    , 1.    , 1.    , 1.    , 1.    ,
        0.9988, 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 1.    , 0.9999], dtype=float32),
 array([0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
        1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
        1, 1]),
 ['cats',
  'cats',
  'cats',
  'dogs',
  'dogs',
  'cats',
  'dogs',
  'dogs',
  'dogs',
  'dogs',
  'cats',
  'dogs',
  'dogs',
  'cats',
  'dogs',
  'cats',
  'dogs',
  'dogs',
  'dogs',
  'dogs',
  'ca

### BUILD VGG FROM SCRATCH

#### REQUIRED IMPORTS

In [20]:
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

In [21]:
FILES_PATH = 'http://files.fast.ai/models/'; CLASS_FILE='imagenet_class_index.json'
# Keras has a get_file() function that downloads - useful
f_path = get_file(CLASS_FILE, FILES_PATH+CLASS_FILE, cache_subdir='models')
with open(f_path) as f: class_dict = json.load(f)
# Convert dictionary with string indexes to an array
classes = [class_dict[str(i)][1] for i in range(len(class_dict))]

##below are examples of stuff we imported
classes[:5]

[u'tench', u'goldfish', u'great_white_shark', u'tiger_shark', u'hammerhead']

### MODEL CREATION

In [31]:
def ConvBlock(layers, model, filters):
    for i in range(layers):
        model.add(ZeroPadding2D((1,1)))
        model.add(Convolution2D(filters, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))    

In [32]:
def FCBlock(model):
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

In [33]:
vgg_mean = np.array([123.68, 116.779, 103.939]).reshape((3,1,1))

def vgg_preprocess(x):
    x = x - vgg_mean
    return x[:, ::-1] ##reverse the axis shorthand

In [34]:
def VGG_16():
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape=(3, 244, 244)))
    
    ConvBlock(2, model, 64)
    ConvBlock(2, model, 128)
    ConvBlock(2, model, 256)
    ConvBlock(2, model, 512)
    ConvBlock(2, model, 512)
    
    model.add(Flatten())
    FCBlock(model)
    FCBlock(model)
    model.add(Dense(1000, activation='softmax'))
    
    return model

In [35]:
model = VGG_16()

In [36]:
fpath = get_file('vgg16.h5', FILES_PATH+'vgg16.h5', cache_subdir='models')
model.load_weights(fpath)

Exception: You are trying to load a weight file containing 16 layers into a model with 13 layers.

In [ ]:
batch_size = 4
